<a href="https://colab.research.google.com/github/colinmcnamara/austin_langchain/blob/main/labs/LangChain_101/Misc/101-1-streamlit_ollama_streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -q langchain streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 9.3 MB/s eta 0:00:00


In [8]:
%%writefile streaming_app.py
from langchain.callbacks.base import BaseCallbackHandler
from langchain.chat_models import ChatOllama
from langchain.schema import HumanMessage, AIMessage
import urllib.request, json
import streamlit as st

@st.cache_resource
def get_ollama_models(ollama_server_url):
    api_tags = "/api/tags"
    models = []
    with urllib.request.urlopen(ollama_server_url + api_tags) as tags:
        response = json.load(tags)
        models = [model['name'] for model in response['models']]

        # This is not necessary but added here to keep the output clean
        models = [model.replace(":latest","") for model in models]
    return tuple(models)

class StreamHandler(BaseCallbackHandler):
    def __init__(self, container, initial_text=""):
        self.container = container
        self.text = initial_text

    def on_llm_new_token(self, token: str, **kwargs) -> None:
        self.text += token
        self.container.markdown(self.text)

st.title("LangChain + Streamlit + Ollama")

with st.sidebar:
    # Url of the hosted ollama instance.
    ollama_server = st.text_input("Ollama API Server", value="http://localhost:11434")
    ollama_model = st.selectbox("Ollama model name", get_ollama_models(ollama_server), index=None)

    st.markdown("""
    Changing the model doesn't clear the history which gets passed to the new llm as context.
    Use the below button to clear chat history and start with a clear slate.
    """)
    if st.button("Clear chat history", type="primary"):
        st.session_state["messages"] = [AIMessage(content="How can I help you?")]


if "messages" not in st.session_state:
    st.session_state["messages"] = [AIMessage(content="How can I help you?")]

for msg in st.session_state.messages:
    st.chat_message(msg.type).write(msg.content)

if prompt := st.chat_input():
    st.session_state.messages.append(HumanMessage(content=prompt))
    st.chat_message("user").write(prompt)

    if not ollama_model:
        st.info("""Please select an existing Ollama model name to continue.
        Visit https://ollama.ai/library for a list of supported models.
        Restart the streamlit app after downloading a model using the `ollama pull <model_name>` command.
        It should become available in the list of available models.""")
        st.stop()

    with st.chat_message("assistant"):
        stream_handler = StreamHandler(st.empty())
        llm = ChatOllama(model=ollama_model, streaming=True, callbacks=[stream_handler])
        response = llm(st.session_state.messages)
        st.session_state.messages.append(AIMessage(content=response.content))

Overwriting streaming_app.py


### Download and run ollama

Below, we:
1. download the ollama binary
2. make it executable
3. start ollama in the background
4. download the hosted mistral model
5. download the hosted llama2 model

In [3]:
%%capture
!curl -L https://ollama.ai/download/ollama-linux-amd64 -o ollama
!chmod +x ollama
!./ollama serve &>/content/ollama_logs.txt &
!./ollama pull mistral
!./ollama pull llama2

### Start and background streamlit app

In [ ]:
!streamlit run streaming_app.py &>/content/logs.txt &

## Find the IP of your instance

In [5]:
!curl ipv4.icanhazip.com
!echo "Copy this IP into the webpage that opens below"

35.227.3.82
Copy this IP into the webpage that opens below


## Expose the Streamlit app on port 8501

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 3.73s
your url is: https://common-mails-doubt.loca.lt
